# 0.0 Import modules

In [110]:
# === Switch to kernel bw25! ===

# Brightway libraries
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw_processing as bwp
import bw2analyzer as ba

# General libraries
import numpy as np
import pandas as pd
from fs.zipfs import ZipFS
import json                        # Library for working with json files
from pathlib import Path           # Library for working with paths in different OS     
import matplotlib.pyplot as plt    # Library for creating plots
from collections import defaultdict

In [111]:
bi.__version__

(0, 9, 'DEV9')

# 0.1 set up project

In [112]:
bd.projects.set_current('ei38-teaching-25')

In [113]:
#if 'ei38-teaching-25' not in bd.projects:
bi.restore_project_directory(r"C:\Users\cko\sciebo1\01_CK\01_Work\1_FCN-ESE\04_Modelle\LCA_Brightway\Autumn_School\brightway2-project-ei38-teaching-25-backup.26-October-2022-03-35PM.tar.gz")

Restoring project backup archive - this could take a few minutes...


'ei38-teaching-25'

In [114]:
bd.databases

Databases dictionary with 4 object(s):
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	biosphere3
	ei 3.8 cutoff

In [115]:
#Instantiating a new project
###
#setup brightway
#bd.projects.set_current('powerplants_LCA')
#bi.bw2setup()

In [116]:
# ### variables

# db_version = "38_cut"


In [117]:
# ## dictonaries for options
# bi.create_core_migrations()

# del bd.Databases['ecoinvent 3.8_cutoff_ecoSpold02']

# db_versions = {"371_cut": 'ecoinvent 3.7.1_cutoff_ecoSpold02', "38_cut": 'ecoinvent 3.8_cutoff_ecoSpold02', "38_cons":'ecoinvent 3.8_consequential_ecoSpold02', "38_cut": 'ecoinvent 3.8_cutoff_ecoSpold02', "38_cut_test":'ecoinvent 3.8_cutoff_ecoSpold02_test'}

# #importing requiered database (ecoinvent):
# #ecoinvent
# if db_versions[db_version] in bi.databases:
#     print("Database has already been imported.")
# else:
#     fpei =r'C:\Users\cko\sciebo1\01_CK\01_Work\1_FCN-ESE\04_Modelle\LCA_Brightway\Databases/'+db_versions[db_version]+'/datasets'
#     ei = bi.SingleOutputEcospold2Importer(fpei, db_versions[db_version])
#     ei
#     ei.apply_strategies()
#     ei.statistics()
#     ei.write_database()



In [118]:
bi.databases

Databases dictionary with 4 object(s):
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	biosphere3
	ei 3.8 cutoff

In [119]:
eiDB = bd.Database('ei 3.8 cutoff')

# 1.Electricity

## 1.1 Import electricity mix and load profile

In [120]:
scenario_year = 2021
weather_year= 2012
sector = 'Chemical & petrochemical'

file_name_em= 'assets/Results_dispatched_power_'+str(scenario_year)+'_'+str(weather_year)+'.csv'
file_name_lp = 'assets/load_profiles_all.csv'


In [121]:
df_elecmix= pd.read_csv(file_name_em, index_col=0, sep=';')
df_load=pd.read_csv(file_name_lp, sep=';')


In [122]:
load_profile_raw = df_load[sector] 
load_profile_raw

0       0.000114
1       0.000114
2       0.000114
3       0.000114
4       0.000114
          ...   
8755    0.000114
8756    0.000114
8757    0.000114
8758    0.000114
8759    0.000114
Name: Chemical & petrochemical, Length: 8760, dtype: float64

In [123]:
df_elecmix=df_elecmix.divide(df_elecmix.sum(axis=1), axis=0)
#df_elecmix_norm_year = df_elecmix/8760

In [124]:
df_elecmix

,bm_st,hy_rs,wi_on,wi_off,so_ro,nu,li_st,hc_cc,ng_cc,hc_st,ng_ce,ng_gt,ng_st,so_op,hy_rr
0,0.117241,0.030946,0.009777,0.101845,0.0,0.209002,0.297888,0.088348,0.144954,0.0,0.0,0.0,0.0,0.0,0.0
1,0.119897,0.031528,0.010351,0.095217,0.0,0.214692,0.305616,0.080203,0.142496,0.0,0.0,0.0,0.0,0.0,0.0
2,0.121032,0.030996,0.008115,0.087003,0.0,0.217028,0.312838,0.081628,0.141360,0.0,0.0,0.0,0.0,0.0,0.0
3,0.121592,0.031603,0.008517,0.075741,0.0,0.218383,0.330441,0.076410,0.137313,0.0,0.0,0.0,0.0,0.0,0.0
4,0.123321,0.031408,0.007940,0.068989,0.0,0.221651,0.337086,0.073802,0.135802,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,0.073612,0.026386,0.096641,0.555646,0.0,0.099422,0.063879,0.038663,0.045751,0.0,0.0,0.0,0.0,0.0,0.0
8756,0.076121,0.026866,0.108360,0.539061,0.0,0.097895,0.066006,0.038701,0.046989,0.0,0.0,0.0,0.0,0.0,0.0
8757,0.080626,0.029598,0.114402,0.522903,0.0,0.091760,0.070275,0.040770,0.049666,0.0,0.0,0.0,0.0,0.0,0.0
8758,0.083902,0.029215,0.117250,0.524191,0.0,0.078956,0.073291,0.041612,0.051583,0.0,0.0,0.0,0.0,0.0,0.0


In [125]:
df_prod_ut=df_elecmix.mul(load_profile_raw, axis=0)

In [126]:
list_pp = df_prod_ut.columns

In [127]:
dyn_amount_pp = df_prod_ut.sum()

In [128]:
share_pp_el_DE_2021 = df_elecmix.sum()/8760

In [129]:
share_pp_el_DE_2021

bm_st     0.084472
hy_rs     0.031765
wi_on     0.048570
wi_off    0.179523
so_ro     0.087157
nu        0.140811
li_st     0.209630
hc_cc     0.108178
ng_cc     0.109780
hc_st     0.000000
ng_ce     0.000000
ng_gt     0.000000
ng_st     0.000000
so_op     0.000000
hy_rr     0.000000
dtype: float64

In [130]:
dyn_amount_pp

bm_st     0.084472
hy_rs     0.031765
wi_on     0.048570
wi_off    0.179522
so_ro     0.087157
nu        0.140811
li_st     0.209630
hc_cc     0.108178
ng_cc     0.109779
hc_st     0.000000
ng_ce     0.000000
ng_gt     0.000000
ng_st     0.000000
so_op     0.000000
hy_rr     0.000000
dtype: float64

In [131]:
# available power plants
dic_powerplants= {'lignite':['li_st'], 'hardcoal':['hc_cc','hc_st'], 'nuclear':['nu'],'naturalgas': ['ng_cc', 'ng_ce', 'ng_gt', 'ng_st'], 'solar': ['so_ro','so_op'], 'wind': ['wi_off','wi_on'],'biomass': ['bm_st'], 'hydro' : ['hy_rs','hy_rr']}

dic_powerplants_market=defaultdict(float)

In [132]:
dic_powerplants_2= {'el_bio_chp':'bm_st',
'el_hydro_reservoir':'hy_rs',
'el_wind_onshore_medium':'wi_on',
'el_wind_offshore':'wi_off',
'el_pv_roof':'so_ro',
'el_nuclear_pressure':'nu'
,'el_lg':'li_st'
,'cc_hc':'hc_cc'
,'el_ng_cc':'ng_cc',
'el_hc':'hc_st',
'el_ng_cc':'ng_cc',
'el_hc':'hc_st',
'el_ng_conv':'ng_ce',
'el_ng_turb':'ng_gt',
'el_ng_conv':'ng_st',
'el_pv_openfield':'so_op',
'el_hydro_runriver':'hy_rr'}

In [133]:
el_hcel_hc = eiDB.search("electricity production, hard coal", filter={'location':"DE"})

Excluding 164 filtered results


In [134]:
#dictonary for all  
dic_el_mix = {}

# hard coal
el_hc = eiDB.search("electricity production, hard coal", filter={'location':"DE"})
el_hc = el_hc[0]
dic_el_mix['el_hc']=el_hc

cc_hc = eiDB.search("heat and power co-generation, hard coal", filter={'location':"DE"})
cc_hc = cc_hc[0]
dic_el_mix['cc_hc']=cc_hc

# lignite
el_lg = eiDB.search("electricity production, lignite", filter={'location':"DE"})
el_lg = el_lg[0]
dic_el_mix['el_lg']=el_lg

# wind
el_wind_onshore_medium = eiDB.search("electricity production, wind, 1-3MW turbine, onshore", filter={'location':"DE"})
el_wind_onshore_medium = el_wind_onshore_medium [0]
dic_el_mix['el_wind_onshore_medium']=el_wind_onshore_medium

el_wind_onshore_small = eiDB.search("electricity production, wind, <1MW turbine, onshore", filter={'location':"DE"})
el_wind_onshore_small = el_wind_onshore_small [0]
dic_el_mix['el_wind_onshore_small']=el_wind_onshore_small

el_wind_onshore_large = eiDB.search("electricity production, wind, >3MW turbine, onshore", filter={'location':"DE"})
el_wind_onshore_large = el_wind_onshore_large [0]
dic_el_mix['el_wind_onshore_large']=el_wind_onshore_large

el_wind_offshore = eiDB.search("electricity production, wind, 1-3MW turbine, offshore", filter={'location':"DE"})
el_wind_offshore =el_wind_offshore [0]
dic_el_mix['el_wind_offshore']=el_wind_offshore

# photovoltaic
el_pv_roof = eiDB.search("electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted", filter={'location':"DE"})
el_pv_roof = el_pv_roof[0]
dic_el_mix['el_pv_roof']=el_pv_roof


el_pv_openfield = eiDB.search("electricity production, photovoltaic, 570kWp open ground installation, multi-Si", filter={'location':"DE"})
el_pv_openfield = el_pv_openfield[0]
dic_el_mix['el_pv_openfield']=el_pv_openfield

## natural gas
el_ng_10MW = eiDB.search("electricity production, natural gas, 10MW", filter={'location':"DE"})
el_ng_10MW =el_ng_10MW [0]
dic_el_mix['el_ng_10MW']=el_ng_10MW

el_ng_cc = eiDB.search("electricity production, natural gas, combined cycle power plant", filter={'location':"DE"})
el_ng_cc=el_ng_cc[0]
dic_el_mix['el_ng_cc']=el_ng_cc

el_ng_conv = eiDB.search("electricity production, natural gas, conventional power plant", filter={'location':"DE"})
el_ng_conv= el_ng_conv[0]
dic_el_mix['el_ng_conv']=el_ng_conv

el_ng_turb = eiDB.search("natural gas, burned in gas turbine, for compressor station", filter={'location':"DE"})
el_ng_turb=el_ng_turb[0]
dic_el_mix['el_ng_turb']=el_ng_turb

# nuclear
el_nuclear_boiling = eiDB.search("electricity production, nuclear, boiling water reactor", filter={'location':"DE"})
el_nuclear_boiling= el_nuclear_boiling[0]
dic_el_mix['el_nuclear_boiling']=el_nuclear_boiling

el_nuclear_pressure = eiDB.search("electricity production, nuclear, pressure water reactor", filter={'location':"DE"})
el_nuclear_pressure=el_nuclear_pressure[0]
dic_el_mix['el_nuclear_pressure']=el_nuclear_pressure

# biomass
el_bio_chp = eiDB.search("heat and power co-generation, biogas, gas engine", filter={'location':"DE"})
el_bio_chp= el_bio_chp[0]
dic_el_mix['el_bio_chp']=el_bio_chp


# hydro
el_hydro_pumpstorage = eiDB.search("electricity production, hydro, pumped storage",filter={'location':"DE"})
el_hydro_pumpstorage=el_hydro_pumpstorage[0]
dic_el_mix['el_hydro_pumpstorage']=el_hydro_pumpstorage

el_hydro_reservoir = eiDB.search("electricity production, hydro, reservoir, non-alpine region",filter={'location':"DE"})
el_hydro_reservoir= el_hydro_reservoir[0]
dic_el_mix['el_hydro_reservoir']=el_hydro_reservoir

el_hydro_runriver = eiDB.search("electricity production, hydro, run-of-river",filter={'location':"DE"})
el_hydro_runriver=el_hydro_runriver[0]
dic_el_mix['el_hydro_runriver']=el_hydro_runriver

# low voltage – energy mix 2020
el_lowvoltage = eiDB.search("market for electricity, low voltage",filter={'location':"DE"}) ##anpassen! 
el_lowvoltage =el_lowvoltage[0]
dic_el_mix['el_lowvoltage']=el_lowvoltage

# high voltage – energy mix 2020
el_highvoltage = eiDB.search("market for electricity, high voltage",filter={'location':"DE"})
el_highvoltage =el_highvoltage[0]
dic_el_mix['el_highvoltage']=el_highvoltage

Excluding 164 filtered results
Excluding 42 filtered results
Excluding 79 filtered results
Excluding 196 filtered results
Excluding 94 filtered results
Excluding 196 filtered results
Excluding 190 filtered results
Excluding 113 filtered results
Excluding 55 filtered results
Excluding 10 filtered results
Excluding 530 filtered results
Excluding 422 filtered results
Excluding 7 filtered results
Excluding 53 filtered results
Excluding 55 filtered results
Excluding 110 filtered results
Excluding 141 filtered results
Excluding 30 filtered results
Excluding 185 filtered results
Excluding 739 filtered results
Excluding 960 filtered results


In [135]:
dic_powerplants_market = defaultdict(float)
for ECO, EMM in dic_powerplants_2.items():
    dic_powerplants_market[dic_el_mix[ECO]] = dyn_amount_pp[EMM] 


In [136]:
dic_powerplants_el_DE_market = defaultdict(float)
for ECO, EMM in dic_powerplants_2.items():
    dic_powerplants_el_DE_market[dic_el_mix[ECO]['name']] = share_pp_el_DE_2021[EMM] 
    print(dic_el_mix[ECO]['name'])


heat and power co-generation, biogas, gas engine
electricity production, hydro, reservoir, non-alpine region
electricity production, wind, >3MW turbine, onshore
electricity production, wind, 1-3MW turbine, offshore
electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted
electricity production, nuclear, pressure water reactor
electricity production, lignite
heat and power co-generation, hard coal
electricity production, natural gas, combined cycle power plant
electricity production, hard coal
electricity production, natural gas, conventional power plant
natural gas, burned in gas turbine, for compressor station
electricity production, photovoltaic, 570kWp open ground installation, multi-Si
electricity production, hydro, run-of-river


In [137]:
#df = pd.DataFrame(dic_powerplants_el_DE_market, index=[0])

In [138]:
# available power plants
dic_powerplants= {'lignite':['li_st'], 'hardcoal':['hc_cc','hc_st'], 'nuclear':['nu'],'naturalgas': ['ng_cc', 'ng_ce', 'ng_gt', 'ng_st'], 'solar': ['so_ro','so_op'], 'wind': ['wi_off','wi_on'],'biomass': ['bm_st'], 'hydro' : ['hy_rs','hy_rr']}

dic_powerplants_market=defaultdict(float)

In [139]:
import yaml

with open('data.yml', 'w') as outfile:
    yaml.dump(dic_powerplants_el_DE_market, outfile, default_flow_style=False)

## 2.0 Replace heat and electricity

In [140]:
### choose variables: 
# electrification: yes/no?
binary_var_electrification = 1 #1=electrification, 0=no electrifaction

#which process?
process_name = 'polyethylene terephthalate production, granulate, bottle grade'
process_loc = 'RER'

method = ('IPCC 2013', 'climate change', 'GWP 100a')

In [141]:
process = eiDB.search(process_name, filter={'location':process_loc})
process = process[0]

Excluding 5 filtered results


In [163]:
def replace_electricity_heat(process_name,process_loc, method, binary_var_electrification):
    process = eiDB.search(process_name, filter={'location':process_loc})
    process = process[0]
    

    dic_el_remove = defaultdict(float)
    dic_heat_remove = defaultdict(float)

    for x in process.edges():
        if 'electricity' in x.input['name']:
            dic_el_remove[x.input] = x['amount']
            
        if 'heat' in x.input['name']:
            dic_heat_remove[x.input] = x['amount']

    el_demand_process = sum([value for value in dic_el_remove.values()])
    heat_demand_process = sum([value for value in dic_heat_remove.values()])


    total_el_demanded = el_demand_process+ binary_var_electrification*heat_demand_process #
    total_heat_demanded = heat_demand_process * (1-binary_var_electrification)


    electrifying_heat = 10_000_000 ##create id dynamically - random number generator

    indices = np.array(
        [
            (electrifying_heat, electrifying_heat), # Production exchange for new motor
            (process.id, electrifying_heat),              
        ] + [
            (node.id, electrifying_heat) for node in dic_el_remove          ##remove existing electricity
        ] + [
            (node.id, electrifying_heat) for node in dic_heat_remove          ##remove existing heat
        ] + [
            (node.id,  electrifying_heat) for node in dic_powerplants_market  ## add Germany electricity mix        ##TODO add German Heat mix
        ], dtype=bwp.INDICES_DTYPE
    )


    data = np.array([
            1,
            1,
        ] + [
            value for value in dic_el_remove.values()       #remove amount of electricity
            
        ] + [
            value for value in dic_heat_remove.values()     #remove amount of heat
            
        ] + [
            value * total_el_demanded for value in dic_powerplants_market.values()    
        ]
    ) 
    flip = np.array(
        [False, True] + [False for _ in dic_el_remove] + [False for _ in dic_heat_remove] + [True for _ in dic_powerplants_market]
        )


    activities = np.array(
        [
            (process, electrifying_heat),              
        ] + [
            (node, electrifying_heat) for node in dic_el_remove          ##remove existing electricity
        ] + [
            (node, electrifying_heat) for node in dic_heat_remove          ##remove existing heat
        ] + [
            (node,  electrifying_heat) for node in dic_powerplants_market  ## add Germany electricity mix        ##TODO add German Heat mix
        ],
    )

    dp = bwp.create_datapackage()

    # dp.add_persistent_vector(
    #     matrix="technosphere_matrix",
    #     data_array=data,
    #     indices_array=indices,
    #     flip_array=flip,
    #     name='electrifying_heat',
    # )

    # _, data_objs, _ = bd.prepare_lca_inputs({process: 1}, method)

    # lca = bc.LCA({process.id: 1}, data_objs=data_objs + [dp])
    # lca.lci()
    # lca.lcia()
    # lca.score
    # print(lca.score)

    # lca.lcia({electrifying_heat: 1})
    # lca.score
    # print(lca.score)

    name_new_process = str('electrification'+process_name+'_'+process_loc)
    


    return ([name_new_process,dp, indices, data, flip, dic_el_remove])

In [164]:
results = replace_electricity_heat(process_name,process_loc, method, binary_var_electrification)
name_new_process=results[0]
dp = results[1]
indices = results[2]
data = results[3]
flip = results[4]
dic_el_remove=results[5]


Excluding 5 filtered results


In [144]:
name_new_db = 'db_electrification'

In [145]:
if name_new_db in bd.databases:
    del bd.databases[name_new_db] 

In [152]:
new_db = bd.Database(name_new_db)

In [147]:
new_db.write({}) ##empty databbase

In [148]:
new_process = new_db.new_node(code="new_process_01", name=name_new_process, location=process_loc, unit="kilogram")
new_process.save()

In [149]:
edges = []
for idx, x in enumerate(indices):
    dic = {'row':x[0], 'column' :x[1], 'amount':data[idx], 'flip':flip[idx]}
    edges.append(dic)


In [165]:
dic_el_remove

defaultdict(float,
            {'market group for electricity, medium voltage' (kilowatt hour, RER, None): 0.198504711728689})

In [ ]:
edges_2 = []
for idx, x in enumerate(indices):
    dic = {'row':x[0], 'column' :x[1], 'amount':data[idx], 'flip':flip[idx]}
    edges.append(dic)

In [ ]:
activities = np.array(
        [
            (process, electrifying_heat),              
        ] + [
            (node, electrifying_heat) for node in dic_el_remove          ##remove existing electricity
        ] + [
            (node, electrifying_heat) for node in dic_heat_remove          ##remove existing heat
        ] + [
            (nod,  electrifying_heat) for node in dic_powerplants_market  ## add Germany electricity mix        ##TODO add German Heat mix
        ],
    )

In [153]:
edges

[{'row': 10000000, 'column': 10000000, 'amount': 1.0, 'flip': False},
 {'row': 18148, 'column': 10000000, 'amount': 1.0, 'flip': True},
 {'row': 23706,
  'column': 10000000,
  'amount': 0.198504711728689,
  'flip': False},
 {'row': 13632,
  'column': 10000000,
  'amount': 0.387656286392523,
  'flip': False}]

In [158]:
for i in edges:
    print(i['row'])
    print(i['amount'])
    new_process.new_edge(input=i['row'], amount=i['amount'], type='technosphere').save()

10000000
1.0


ValueError: Provided input data is invalid

In [151]:
#new_db.new_(edges, [],  ["ei 3.8 cutoff"])

AttributeError: 'SQLiteBackend' object has no attribute 'write_exchanges'

In [ ]:
process.new_edge(input=new_process, amount=0, type="technosphere").save()

In [159]:
new_process

'electrificationpolyethylene terephthalate production, granulate, bottle grade_RER' (kilogram, RER, None)

In [ ]:
_, data_objs, _ = bd.prepare_lca_inputs({process: 1}, method)

In [ ]:
lca = bc.LCA({process.id: 1}, data_objs=data_objs)
lca.lci()
lca.lcia()
lca.score

NonsquareTechnosphere: Technosphere matrix is not square: 19566 activities (columns) and 19567 products (rows). Use LeastSquaresLCA to solve this system, or fix the input data

In [ ]:
lca.lcia({new_process.id: 1})
lca.score

In [ ]:
bd.Database("ei 3.8 cutoff").metadata['depends']

['biosphere3']

In [ ]:
bd.Database(new_db).metadata['depends']

UnknownObject: This object is not yet registered; can't get or set metadata